Attempt at adding a layer to the previous ResNet model created following [kaggle notebook ResNet for MNIST with PyTorch](https://www.kaggle.com/readilen/resnet-for-mnist-with-pytorch?scriptVersionId=6942243) and the [PyTorch Tutorial on implementation of a ResNet model](https://pytorch-tutorial.readthedocs.io/en/latest/tutorial/chapter03_intermediate/3_2_2_cnn_resnet_cifar10/) to see what happens, if the accuracy increases. Code is due to [Liu Kuangs's extensive code](https://github.com/kuangliu).

***
<span style="color: blue"><b>Study session observations: </b></span>

#### 20/07/2020:
Had difficulties implementing a 4th layer due to size, during forward propagation of residual block. Will look more into that after a more concise understanding of the archictecture and classes used. Changed the notebook's name in order to be more incisive with the notebook content. 

Will attempt to change a few input values of certain variables (like learning_rate, train and test loader batch size, etc.) and also both forward methods to see what happens. 

Also want to see more about num_epochs needed and in general more about epochs. When using 80 epochs, the accuracy was of ~10-11%. With 1, ~9-10%. Want to see what can influence the accuracy of the model.
***

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

### Dataset
***

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [24]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()])

In [25]:
train_dataset = torchvision.datasets.CIFAR10(root='../cifar-10-batches-py/',
                                             train=True, 
                                             transform=transform)

test_dataset = torchvision.datasets.CIFAR10(root='../cifar-10-batches-py/',
                                            train=False, 
                                            transform=transforms.ToTensor())

In [31]:
print(train_dataset.data.shape)

(50000, 32, 32, 3)


In [26]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=100, 
                                           shuffle=True) #Original tutorial has shuffle=True

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=100, 
                                          shuffle=False)

### Creating classes and functions related to ResNet

In [16]:
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels,
                     out_channels,
                     kernel_size=3,
                     stride=stride,
                     padding=1,
                     bias=False)

In [56]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample
    
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        
        if self.downsample:
            residual = self.downsample(x)
            
        out += residual
        out = self.relu(out)
        
        return out

In [57]:
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv = conv3x3(1, 16)  # 1 when using mnist, 3 when using cifar10
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        
        print("\nlayer1")
        self.layer1 = self.make_layer(block, 16, layers[0])
        print("\nlayer2")
        self.layer2 = self.make_layer(block, 32, layers[1], 2)
        print("\nlayer3")
        self.layer3 = self.make_layer(block, 64, layers[2], 2)
#         print("\nlayer4")
#         self.layer4 = self.make_layer(block, 128, layers[3], 2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)
        
    
    def make_layer(self, block, out_channels, blocks, stride=1):
        
        downsample = None
        
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                conv3x3(self.in_channels, out_channels, stride=stride),
                nn.BatchNorm2d(out_channels))
        
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
#         out = self.layer4(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)  # unsure what this is
        out = self.fc(out)
        
        return out

### Training model
***

In [72]:
def train_model(train_loader, model, num_epochs, error, optimizer, curr_lr, device):
    print("Training model")
    
    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(train_loader):
            images = images.resize_(100, 1, 32, 32).to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = error(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if (i+1) % 100 == 0:
                print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}"
                       .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

        # Decay learning rate
        if (epoch+1) % 20 == 0:
            curr_lr /= 3
            update_lr(optimizer, curr_lr)

        print("Current learning rate: ", curr_lr)
        
    return model

In [79]:
net_args = {
    "block" : ResidualBlock,
    "layers": [2, 2, 2, 2]
}

model = ResNet(**net_args).to(device)


layer1

layer2

layer3


In [74]:
num_epochs = 10
learning_rate = 0.001

In [75]:
error = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [67]:
def update_lr(optimizer, lr):    
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [68]:
total_step = len(train_loader)
curr_lr = learning_rate

In [81]:
model = train_model(train_loader, model, 1, error, optimizer, curr_lr, device)

Training model
Epoch [1/1], Step [100/500] Loss: 2.3603
Epoch [1/1], Step [200/500] Loss: 2.3389
Epoch [1/1], Step [300/500] Loss: 2.4024
Epoch [1/1], Step [400/500] Loss: 2.2993
Epoch [1/1], Step [500/500] Loss: 2.3834
Current learning rate:  0.001


### Testing model
***

In [82]:
def test_model(model, test_loader, device): 
    print("Testing model")
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.resize_(100, 1, 32, 32).to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))

In [84]:
test_model(model, test_loader, device)

Testing model
Accuracy of the model on the test images: 9.62 %
